<a href="https://colab.research.google.com/github/felolivee/DVA-NYC_Congestion/blob/Ashley/DVA_Project_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download from Kaggle
! pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d jeffsinsel/nyc-fhvhv-data

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/jeffsinsel/nyc-fhvhv-data
License(s): CC0-1.0
100% 17.8G/17.8G [03:53<00:00, 52.9MB/s]
100% 17.8G/17.8G [03:53<00:00, 81.9MB/s]


In [2]:
#unzip files in nyc-fhvhv-data from Kaggle
! unzip nyc-fhvhv-data.zip

Archive:  nyc-fhvhv-data.zip
  inflating: data_dictionary_trip_records_hvfhs.pdf  
  inflating: fhvhv_tripdata_2019-02.parquet  
  inflating: fhvhv_tripdata_2019-03.parquet  
  inflating: fhvhv_tripdata_2019-04.parquet  
  inflating: fhvhv_tripdata_2019-05.parquet  
  inflating: fhvhv_tripdata_2019-06.parquet  
  inflating: fhvhv_tripdata_2019-07.parquet  
  inflating: fhvhv_tripdata_2019-08.parquet  
  inflating: fhvhv_tripdata_2019-09.parquet  
  inflating: fhvhv_tripdata_2019-10.parquet  
  inflating: fhvhv_tripdata_2019-11.parquet  
  inflating: fhvhv_tripdata_2019-12.parquet  
  inflating: fhvhv_tripdata_2020-01.parquet  
  inflating: fhvhv_tripdata_2020-02.parquet  
  inflating: fhvhv_tripdata_2020-03.parquet  
  inflating: fhvhv_tripdata_2020-04.parquet  
  inflating: fhvhv_tripdata_2020-05.parquet  
  inflating: fhvhv_tripdata_2020-06.parquet  
  inflating: fhvhv_tripdata_2020-07.parquet  
  inflating: fhvhv_tripdata_2020-08.parquet  
  inflating: fhvhv_tripdata_2020-09.parquet

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
import glob

# initialize spark
spark = SparkSession.builder.appName("NYC_Rides").getOrCreate()

# list of all parquet files from content folder
parquet_files = glob.glob('/content/*.parquet')

removed_summary = {}
combined_df = None

for file in parquet_files:
    # read parquet files get features
    df = spark.read.parquet(file).select("Hvfhs_license_num", "Pickup_datetime", "DropOff_datetime", "PULocationID", "DOLocationID", "base_passenger_fare", "trip_miles", "tips", "driver_pay", "trip_time")
    initial_count = df.count()

    # clean data
    df_clean = df.filter(
        (F.col("Pickup_datetime").isNotNull()) &
        (F.col("DropOff_datetime").isNotNull()) &
        (F.col("PULocationID").isNotNull()) &
        (F.col("DOLocationID").isNotNull()) &
        (F.col("base_passenger_fare").isNotNull()) &
        (F.col("trip_miles").isNotNull()) &
        (F.col("tips").isNotNull()) &
        (F.col("driver_pay").isNotNull()) &
        (F.col("trip_time").isNotNull()) &
        (F.col("Hvfhs_license_num").isNotNull()) &
        (F.col("base_passenger_fare") > 0) &
        (F.col("trip_miles") > 0) &
        (F.col("driver_pay") > 0) &
        (F.col("trip_time") > 0)
    )
    clean_count = df_clean.count()
    removed_summary[file] = initial_count - clean_count

    # combine clean DFs into one big DF
    if combined_df is None:
        combined_df = df_clean
    else:
        combined_df = combined_df.union(df_clean)

# check how many of which file were removed
print("Removal Summary:", removed_summary)

Removal Summary: {'/content/fhvhv_tripdata_2021-07.parquet': 140020, '/content/fhvhv_tripdata_2020-12.parquet': 104215, '/content/fhvhv_tripdata_2020-11.parquet': 103002, '/content/fhvhv_tripdata_2021-01.parquet': 103408, '/content/fhvhv_tripdata_2020-08.parquet': 135659, '/content/fhvhv_tripdata_2022-10.parquet': 159570, '/content/fhvhv_tripdata_2021-05.parquet': 142950, '/content/fhvhv_tripdata_2020-02.parquet': 2034127, '/content/fhvhv_tripdata_2022-02.parquet': 84755, '/content/fhvhv_tripdata_2022-03.parquet': 84930, '/content/fhvhv_tripdata_2022-04.parquet': 82929, '/content/fhvhv_tripdata_2020-07.parquet': 188921, '/content/fhvhv_tripdata_2021-12.parquet': 94795, '/content/fhvhv_tripdata_2021-03.parquet': 140061, '/content/fhvhv_tripdata_2020-09.parquet': 132524, '/content/fhvhv_tripdata_2022-07.parquet': 77160, '/content/fhvhv_tripdata_2022-09.parquet': 85805, '/content/fhvhv_tripdata_2022-06.parquet': 87529, '/content/fhvhv_tripdata_2020-03.parquet': 949378, '/content/fhvhv_tri

In [4]:
# calculate how many total rows of data were removed
tot = 0
for key,val in removed_summary.items():
  tot += int(val)

count = combined_df.count()
percent_removed = tot/count * 100

print("{}% was removed from a total of {} rides in 46 months".format(percent_removed, count))

5.159562385472075% was removed from a total of 708720164 rides in 46 months


In [ ]:
# show schema
combined_df.printSchema()

root
 |-- Hvfhs_license_num: string (nullable = true)
 |-- Pickup_datetime: timestamp_ntz (nullable = true)
 |-- DropOff_datetime: timestamp_ntz (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- trip_time: long (nullable = true)



In [6]:
# aggregate features for Tableau file
add_hour = combined_df.withColumn("hour", F.date_format(F.col("Pickup_datetime"), "k"))

tableau_df = add_hour.groupBy(["PULocationID", "hour", "Hvfhs_license_num"]).agg(F.count("PULocationID").alias("ride_count"),F.mean("trip_miles").alias("avg_trip_miles"))

tableau_df.show()

+------------+----+-----------------+----------+------------------+
|PULocationID|hour|Hvfhs_license_num|ride_count|    avg_trip_miles|
+------------+----+-----------------+----------+------------------+
|         114|  24|           HV0003|    238711| 4.210911562517022|
|         249|  24|           HV0005|     90121| 4.106616548862087|
|           7|  24|           HV0005|     69393| 4.878286008675229|
|         155|   1|           HV0003|     22783| 5.026119475047185|
|          28|   1|           HV0005|      7988| 6.039037806710063|
|         203|   2|           HV0003|     11726| 6.711203308886236|
|         131|   2|           HV0005|      3384| 6.555192080378253|
|         226|   4|           HV0003|     46809| 5.623231643487364|
|          73|   4|           HV0003|      8730| 4.904871706758304|
|          31|   4|           HV0003|      1810| 5.498077348066297|
|         195|   5|           HV0003|      9658| 7.333329881963142|
|         220|   6|           HV0003|     45272|

In [8]:
tableau_df.write.csv('dva_project.csv')

In [10]:
files.download('/content/dva_project.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>